In [ ]:
import pandas as pd
import numpy as np

# Load dataset with health scores
df = pd.read_csv('../amazon_sales_with_health_scores.csv')

df['delivery_date'] = pd.to_datetime(df['delivery_date'], errors='coerce')
df['data_collected_at'] = pd.to_datetime(df['data_collected_at'], errors='coerce')

categorical_cols = ['product_category','is_best_seller','is_sponsored','has_coupon','buy_box_availability']
for col in categorical_cols:
    df[col] = df[col].astype('category')

print(df.info())

In [ ]:
import pandas as pd

# Define low sales threshold for identifying low sales products
low_sales_threshold = 50

# Group by product category
category_summary = df.groupby("product_category").agg(
    Total_Sales=pd.NamedAgg(column="purchased_last_month", aggfunc=lambda x: (x * df.loc[x.index, "discounted_price"]).sum()),
    Avg_Rating=pd.NamedAgg(column="product_rating", aggfunc="mean"),
    Avg_Health_Score=pd.NamedAgg(column="Health_Score", aggfunc="mean"),
    Total_Products=pd.NamedAgg(column="product_title", aggfunc="count"),
    Low_Sales_Count=pd.NamedAgg(column="purchased_last_month", aggfunc=lambda x: (x <= low_sales_threshold).sum())
).reset_index()

# Calculate Low Sales %
category_summary["Low_Sales_%"] = (category_summary["Low_Sales_Count"] / category_summary["Total_Products"] * 100).round(2)


category_summary.to_csv("category_summary_for_dashboard.csv", index=False)


In [ ]:
# Define thresholds
low_sales_threshold = 50
low_rating_threshold = 3.5  # rating below 3.5 considered low

# Filter dead listings
dead_listings = df[(df['purchased_last_month'] <= low_sales_threshold) & 
                   (df['product_rating'] <= low_rating_threshold)]

# Select relevant columns for dashboard
dead_listings_dashboard = dead_listings[[
    'product_title', 'product_category', 'purchased_last_month', 
    'product_rating', 'discounted_price', 'Health_Score'
]]

# Save to CSV
dead_listings_dashboard.to_csv("dead_listings_for_dashboard.csv", index=False)

# Summary
print(f"Total dead listings: {len(dead_listings)}")
dead_listings_dashboard.head()


In [ ]:
# Checking overcrowded categories
overcrowded_threshold = 3000
category_counts = df['product_category'].value_counts()
overcrowded_categories = category_counts[category_counts > overcrowded_threshold].index.tolist()

# Filter products in overcrowded categories
overcrowded_products = df[df['product_category'].isin(overcrowded_categories)]

# Prepare CSV for dashboard
overcrowded_products_dashboard = overcrowded_products[[
    'product_title', 'product_category', 'purchased_last_month', 
    'product_rating', 'Health_Score', 'discounted_price'
]]
overcrowded_products_dashboard.to_csv("overcrowded_categories_for_dashboard.csv", index=False)
print(f"Overcrowded categories: {overcrowded_categories}")
print(f"Total products in overcrowded categories: {len(overcrowded_products)}")

# High Potential Listings
# Criteria: high health score (>=0.8) and moderate sales (>= median sales)
median_sales = df['purchased_last_month'].median()
high_potential_listings = df[(df['Health_Score'] >= 0.8) & 
                             (df['purchased_last_month'] >= median_sales)]

# Prepare CSV for dashboard
high_potential_dashboard = high_potential_listings[[
    'product_title', 'product_category', 'purchased_last_month', 
    'product_rating', 'Health_Score', 'discounted_price'
]]
high_potential_dashboard.to_csv("high_potential_listings_for_dashboard.csv", index=False)
print(f"Total high-potential listings: {len(high_potential_listings)}")
high_potential_dashboard.head()


In [ ]:
# --- Category-Level KPI Summary for Dashboard ---
category_summary = df.groupby("product_category").agg(
    Total_Products=('product_title', 'count'),
    Total_Sales=('purchased_last_month', 'sum'),
    Avg_Rating=('product_rating', 'mean'),
    Avg_Health_Score=('Health_Score', 'mean'),
    Low_Sales_Count=('purchased_last_month', lambda x: (x < 50).sum()),
    Dead_Listings_Count=('Health_Score', lambda x: ((x < 0.5) & (df.loc[x.index, 'purchased_last_month'] < 50)).sum()),
    High_Potential_Count=('Health_Score', lambda x: ((x >= 0.8) & (df.loc[x.index, 'purchased_last_month'] >= df['purchased_last_month'].median())).sum())
).reset_index()

# Calculate percntages
category_summary['Low_Sales_%'] = (category_summary['Low_Sales_Count'] / category_summary['Total_Products'] * 100).round(2)
category_summary['Dead_Listings_%'] = (category_summary['Dead_Listings_Count'] / category_summary['Total_Products'] * 100).round(2)
category_summary['High_Potential_%'] = (category_summary['High_Potential_Count'] / category_summary['Total_Products'] * 100).round(2)

# Save CSV for dashboard
category_summary.to_csv("category_kpis_for_dashboard.csv", index=False)

category_summary.head()
